In [1]:
import qiskit as qk #quantum computing library
from qiskit.quantum_info import Statevector #Statevector is a vector in the 2**n dimensional complex Hilbert space. It has coefficients.
import numpy as np

KeyboardInterrupt: 

1. Code to create the quantum circuits for each step, to change circuits & to change measurements.

In [ ]:
kR = np.pi/4.0 #rotation angle for the rotation gates

#gate labels: 0 = I, 1 = H, 2 = X, 3 = Rup, 4 = Rdown, 5 = CX1, 6 = CX2

def createCircuit(data): #function that creates a circuit using the gates selected in the Launchpad. 
    circ = qk.QuantumCircuit(3)
    for i in range(3):
        #add X gates
        if data[i] == 2: #the data is a list of 3 integers, each one representing a qubit. The integers are the gate labels. 
            circ.x(i)
        #add H gates
        if data[i] == 1:
            circ.h(i)
        #add CX gates. The CX gate is a two-qubit gate, so we need to search for the other qubit.
        if data[i] == 5:
            #search for the other qubit
            for j in range(3):
                if data[j] == 6:
                    circ.cx(i, j)
                    break
        
        #add rotation gates. Gate Rz is a rotation around the z axis (vertical axis). That's phi in the Bloch sphere.
        if data[i] == 3: #rotation gate for the up state
            circ.rz(kR, i) 
        if data[i] == 4: #rotation gate for the down state
            circ.rz(-kR, i)
    return circ

def changeCircuit(data, i): #function that changes/updates a circuit in the circuits list
    global circuits
    circuits[i] = createCircuit(data)

def changeMeasure(nqubit, step): #function that changes/updates the measure matrix
    global MM
    #print(nqubit, step)
    MM[step][nqubit] = not MM[step][nqubit] #this acts like a toggle switch within the annotation/measure mode in the Launchpad.

2. Code to define measurements and annotations.

In [ ]:
#This is an example of a state:
#            0        1        2        3         4         5         6         7
# state = 0 |000>+ 2 |001>+ 5 |010>+ 2 |011> + 4 |100> + 6 |101> + 2 |110> + 0 |111>  (notation: |q0q1q2>)
#The labels above are the decimal numbers correspponding to the binary numbers of the basis vectors. (Ex: 011 binary -> 3 decimal)
#These are the coefficients: [0, 2, 5, 2, 4, 6, 2, 0]
#We want this function: getCoeff(state, 011) -> 2     (state[3] = 2)

def getCoeff(_state, b3, b2, b1): #function that returns the coefficient of a state
    index = int(b3 + 2*b2 + 4*b1) #convert binary to decimal
    return _state.data[index]

def extract(A, M1, M2): #A, M1 and M2 are the boolean values of a step in the MM measure matrix. This function annotates/measures if told to do show so through boolen values in each step. 
    global quantumState #current complex vector of coefficients

    #q1 wave function has to be collapsed (measured) if we want to either annonate q0 or measure q1.
    #The entanglement induces hyperspheres and we can no longer anonnate the angle of Bloch unless we collapse q1.
    #When q1 collapses, q0 looses some information and goes back to a 2D space, so that we can annotate the Bloch's angle.
    #The same exact process applies to q2.

    bM1 = A or M1 #boolean value. If A or M1 are true, then bM1 is true and has to be collapsed.
    bM2 = A or M2 #boolean value. If A or M2 are true, then bM2 is true and has to be collapsed.

    rA = -1 #variable that stores the result of the annotation value
    rM1 = -1 #variable that stores the result of the measurement of q1
    rM2 = -1 #same as RM1 but for q2
    rCM1 = -1 #variable that stores the result of the collapsed q1 when we annotate q0
    rCM2 = -1 #same as rCM1 but for q2

    #first, collapse q1 if needed, and store the value if needed
    if bM1:
        (rCM1, stateM1) = quantumState.measure([1]) #collapse second qubit
        if M1:
            rM1 = int(rCM1)
        quantumState = stateM1

    #then, collapse q2 if needed, and store the value if needed
    if bM2:
        (rCM2, stateM2) = (rCM1, stateM1) = quantumState.measure([2])
        if M2:
            rM2 = int(rCM2)
        quantumState = stateM2

    #finally, if we want to anotate, get resulting coeffs and calculate bloch angles
    if A:

        #when q1 and q2 are collapsed, the Bloch sphere remaining is in the space c0|0 rCM1 rCM2> + c1|1 rCM1 rCM2>
        c0 = getCoeff(quantumState, 0, int(rCM1), int(rCM2))
        c1 = getCoeff(quantumState, 1, int(rCM1), int(rCM2))

        #get bloch angles from coefficients c0 and c1, where
        # |q0> = c0|0> + c1|1>
        phi = np.angle(c1)-np.angle(c0) #angle() returns the phase of a complex number
        if c1==0.0 or c0==0.0: #if c1 or c0 are zero, then the angle needs a scale factor of 2
            phi *= 2.0

        #quantize phi (it is a multiple of pi/8 = kR) to avoid precision errors
        rA = int(np.round(phi/(kR)))
        #make it positive
        if rA < 0:
            rA += 8
    
    return (rA, rM1, rM2)


3. Code to fill the circuits and the MM variables.

In [ ]:
circuits = [createCircuit([0,0,0]) for i in range(16)] #list of circuits full with empty values or 0s, which are identity gates, that is updated in each step

MM = [[False for i in range(3)] for j in range(16)] #matrix of False boolean values that is updated in each step

MG =  [[0 for i in range(3)] for j in range(16)] #matrix of gate labels that is updated in each step

quantumState = Statevector.from_label('000') #initial state